In [328]:
import csv
import json
import os
import operator
from itertools import compress 
import numpy as np
import copy 

# loading human generated questions, answers, paragraphs from squad

In [329]:
root_dir = '/Users/tonytu/desktop/old_stuff/berkeley/fall2020/research/CausalQG'
def load(dataset):
    filepath = os.path.join(root_dir, dataset)
    with open(filepath) as file:
        data = json.load(file)
    print('succesfully loaded {} articles from squad dataset'.format(len(data['data'])))
    print(filepath)
    return data['data']
    #print data by paragraphs
    
def parse(data):
    #extract all paragraphs from json 
    titles = []
    paragraphs = []
    questions = []
    final_questions = []
    answers = []
    for i in range(len(data)):
        title = data[i]['title']
        titles.append(title)
        for j in range(len(data[i]['paragraphs'])):
            if j == 0:
                paragraphs.append([])
                questions.append([])
                answers.append([])
#             print(data[i]['paragraphs'][j]['qas'])
            questions[-1].append(data[i]['paragraphs'][j]['qas'])
            answers[-1].append(data[i]['paragraphs'][j]['qas'])
            paragraphs[-1].append(data[i]['paragraphs'][j]['context'])
    for i in range(len(questions)):
        for j in range(len(questions[i])):
            temp = questions[i][j]
#             print(temp)
            questions[i][j] = []
            answers[i][j] = []
            for k in range(len(temp)):
                questions[i][j].append(temp[k]['question']) 
                if len(temp[k]['answers']) != 0:
                    answers[i][j].append(temp[k]['answers'][0])
                else:
                    answers[i][j].append('')
    return titles, paragraphs, questions, answers

squad_2_dev = load('dev-v2.0.json')
squad_2_train = load('train-v2.0.json')
squad_1_dev = load('dev-v1.1.json')
squad_1_train = load('train-v1.1.json')
# squad_2_dev = json.load()
# squad_2_train
# squad_1_dev
# squad_1_train
titles, paragraphs, questions, answers = parse(squad_2_train)
assert len(paragraphs) == len(questions) == len(answers)
len(questions)

succesfully loaded 35 articles from squad dataset
/Users/tonytu/desktop/old_stuff/berkeley/fall2020/research/CausalQG/dev-v2.0.json
succesfully loaded 442 articles from squad dataset
/Users/tonytu/desktop/old_stuff/berkeley/fall2020/research/CausalQG/train-v2.0.json
succesfully loaded 48 articles from squad dataset
/Users/tonytu/desktop/old_stuff/berkeley/fall2020/research/CausalQG/dev-v1.1.json
succesfully loaded 442 articles from squad dataset
/Users/tonytu/desktop/old_stuff/berkeley/fall2020/research/CausalQG/train-v1.1.json


442

# Heuristics (binary classification) 

In [330]:
def longest_common_substring(s1, s2):
    m = [[0] * (1 + len(s2)) for i in range(1 + len(s1))]
    longest, x_longest = 0, 0
    for x in range(1, 2):
        for y in range(1, 1 + len(s2)):
            if s1[x - 1] == s2[y - 1]:
                m[x][y] = m[x - 1][y - 1] + 1
                if m[x][y] > longest:
                    longest = m[x][y]
                    x_longest = x
            else:
                m[x][y] = 0
    return s1[x_longest - longest: x_longest]

def longest_common_opener(s1, s2):
    common_string = ''
    s1_split = s1.split(" ")
    s2_split = s2.split(" ")
    for i in range(len(s1_split)):
        if s1_split[i] == s2_split[i]:
            common_string += s1_split[i]
            common_string += " "
            continue
        else:
            break
    return common_string

def longest_common_sentence(s1, s2):
    s1_words = s1.split(' ')
    s2_words = s2.split(' ')  
    return ' '.join(longest_common_substring(s1_words, s2_words))

def extract_sentences(file_path):
#     with open('textbooktony.csv', 'r') as file:
    with open(file_path, 'r') as file:
        texts = []
        causes = []
        effects = []
        cause_questions = []
        effect_questions = []
        reader = csv.reader(file)
        for row in reader:
            text = row[2]
            cause = row[3]
            effect = row[4]
            cause_question = row[5]
            effect_question = row[6]
            texts.append(text)
            causes.append(cause)
            effects.append(effect)
            cause_questions.append(cause_question)
            effect_questions.append(effect_question)
    return texts, causes, effects, cause_questions, effect_questions

s1 = 'this is a foo bar sentence .'
s2 = 'this is what a kappa foo bar black sheep ?'
# common_sentence = longest_common_sentence(s1, s2)
common_sentence = longest_common_opener(s1, s2)
print (common_sentence)

this is 


In [331]:
def CountFrequency(my_list): 
    # Creating an empty dictionary
    freq = {}
    for item in my_list:
        if (item in freq):
            freq[item] += 1
        else: 
            freq[item] = 1
    return freq
    for key, value in freq.items():
        print ((key, value))

## Most common sentence openers from squad dataset (generated from prophetnet)

In [332]:
# texts, causes, effects, cause_questions, effect_questions = extract_sentences('CSVs/SQuAD/squad_ce_prophetnet.csv')
# cause_questions.extend(effect_questions)

In [429]:
#compare every distinct pair of question 
# common_substrings = []
# for i in range(len(cause_questions)-1):
#     for j in range(i+1, len(cause_questions)):
#         common_sentence = longest_common_opener(cause_questions[i], cause_questions[j])
#         common_substrings.append(common_sentence)
common_substrings = []
for i in range(len(cause_questions)-1):
    for j in range(i+1, len(cause_questions)):
        common_sentence = longest_common_opener(cause_questions[i], cause_questions[j])
        common_substrings.append(common_sentence)

IndexError: list index out of range

In [334]:
cause_counts = CountFrequency(common_substrings)
cause_frequencies = sorted(cause_counts.items(),key=operator.itemgetter(1),reverse=True)
cause_frequencies

[('', 1706450),
 ('what ', 541346),
 ('why ', 68695),
 ('how ', 39972),
 ('what is ', 24049),
 ('what is the ', 12533),
 ('what did ', 9081),
 ('why did ', 5501),
 ('what was the ', 3546),
 ('what happens ', 3340),
 ('why is ', 3260),
 ('what was ', 3033),
 ('when ', 2592),
 ('what does ', 2196),
 ('what did the ', 2099),
 ('who ', 1795),
 ('how did ', 1555),
 ('what are ', 1257),
 ('why did the ', 1225),
 ('how can ', 948),
 ('how do ', 800),
 ('why are ', 763),
 ('what happens if ', 759),
 ('why was ', 734),
 ('what do ', 680),
 ('how is ', 466),
 ('where ', 462),
 ('why do ', 418),
 ('what does the ', 401),
 ('how does ', 400),
 ('what can ', 373),
 ('why does ', 359),
 ('what happens when ', 346),
 ('what is a ', 345),
 ('why is the ', 320),
 ('what has ', 311),
 ('what are the ', 298),
 ('are ', 292),
 ('why were ', 269),
 ('is ', 267),
 ('why was the ', 251),
 ('what would ', 248),
 ('when did ', 232),
 ('how many ', 204),
 ('what happened ', 192),
 ('what causes ', 163),
 ('how 

## Grouping

In [335]:
def classifier(frequencies):
    #binary classifier used to classify the questions as causal
    #or non-causal, if the question starts with why, or any variation
    #of what happens, or contain the token "effect" or "cause", it will
    #be classified as a causal question, otherwise non-causal.
    openers = []
    for i in range(len(frequencies)):
        openers.append(frequencies[i][0])
    cutoff = int(len(frequencies) / 3)
    most_common_openers = openers[:cutoff]
    
    #classification
    causal_openers = []
    non_causal_openers = []
    unclassified = []
    for opener in most_common_openers:
        if opener.lower().startswith('why'):
            causal_openers.append(opener)
            #if opener does not start with why 
        elif opener.lower().startswith('what happens') or \
            opener.lower().startswith('what would happen') or \
            opener.lower().startswith('what will happen') or \
            opener.lower().startswith('what happened'):
            causal_openers.append(opener)
        elif opener.lower().find('effect') != -1 or opener.lower().find('cause') != -1:
            causal_openers.append(opener)
        elif opener.lower().startswith('how') or opener.lower().startswith('what') or opener.lower().startswith('when') or opener.lower().startswith('where') or opener.lower().startswith('who'):
            non_causal_openers.append(opener)
        else:
            unclassified.append(opener)
#     for i in range(20):
#         causal_openers.append(frequencies[i][0])
    return causal_openers, non_causal_openers, unclassified

causal_openers, non_causal_openers, unclassified = classifier(cause_frequencies)

In [336]:
causal_openers = causal_openers[1:]
non_causal_openers = non_causal_openers[1:]
unclassified = unclassified[1:]

# Using the heuristic to filter the human-generated causal questions from the non-causal in squad dataset

In [338]:
import json
import os
root_dir = '/Users/tonytu/desktop/old_stuff/berkeley/fall2020/research/causalQG'
def load(dataset):
    filepath = os.path.join(root_dir, dataset)
    with open(filepath) as file:
        data = json.load(file)
    return data['data']
    #print data by paragraphs
    print('succesfully loaded {} articles from squad dataset'.format(len(data['data'])))
    print(filepath)
#     return data['data']

def parse(data):
    #extract all paragraphs from json 
    titles = []
    paragraphs = []
    questions = []
    final_questions = []
    answers = []
    for i in range(len(data)):
        title = data[i]['title']
        titles.append(title)
        for j in range(len(data[i]['paragraphs'])):
            if j == 0:
                paragraphs.append([])
                questions.append([])
                answers.append([])
#             print(data[i]['paragraphs'][j]['qas'])
            questions[-1].append(data[i]['paragraphs'][j]['qas'])
            answers[-1].append(data[i]['paragraphs'][j]['qas'])
            paragraphs[-1].append(data[i]['paragraphs'][j]['context'])
    #padding questions with no answers with empty strings
    for i in range(len(questions)):
        for j in range(len(questions[i])):
            temp = questions[i][j]
#             print(temp)
            questions[i][j] = []
            answers[i][j] = []
            for k in range(len(temp)):
                questions[i][j].append(temp[k]['question']) 
                if len(temp[k]['answers']) != 0:
                    answers[i][j].append(temp[k]['answers'][0])
                else:
                    answers[i][j].append('')
    #multiplying the paragraph array with the number of questions
    #comment out if no need to duplicate
    for i in range(len(questions)):
        for j in range(len(questions[i])):
            temp = paragraphs[i][j]
            paragraphs[i][j] = [temp] * len(questions[i][j])
    return titles, paragraphs, questions, answers
titles, paragraphs, questions, answers = parse(squad_2_train)


In [339]:
#filtering out the non-causal sentences
from collections.abc import Sequence

def get_shape(lst, shape=()):
    """
    returns the shape of nested lists similarly to numpy's shape.

    :param lst: the nested list
    :param shape: the shape up to the current recursion depth
    :return: the shape including the current depth
            (finally this will be the full depth)
    """
    if not isinstance(lst, Sequence):
        # base case
        return shape
    # peek ahead and assure all lists in the next depth
    # have the same length
    if isinstance(lst[0], Sequence):
        l = len(lst[0])
        if not all(len(item) == l for item in lst):
            msg = 'not all lists have the same length'
            raise ValueError(msg)
    shape += (len(lst), )
    # recurse
    shape = get_shape(lst[0], shape)
    return shape

def is_causal(question, causal_openers):
    return any(question.lower().startswith(opener) for opener in causal_openers)

#get indices of questions which are causal
def get_indices(paragraphs, questions, causal_openers):
    indices = copy.deepcopy(paragraphs)
    for i in range(len(paragraphs)):
        for j in range(len(paragraphs[i])):
            for k in range(len(questions[i][j])):
                if is_causal(questions[i][j][k], causal_openers):
#                     print(questions[i][j][k])
                    indices[i][j][k] = True
                else:
                    indices[i][j][k] = False
    return indices

def flatten(t):
    #flattening list
    flat_list = [item for sublist in t for item in sublist]
    return flat_list
    
indices = get_indices(paragraphs, questions, causal_openers)

In [341]:
paragraphs = flatten(flatten(paragraphs))
questions = flatten(flatten(questions))
answers = flatten(flatten(answers))
indices = flatten(flatten(indices))
# indices = flatten(flatten(indices))
# causal_questions = list(compress(questions, indices))
# causal_paragraphs = list(compress(paragraphs, indices))
# causal_answers = list(compress(answers, indices))


In [416]:
count = len(questions)
causal_count = 0
non_causal_count = 0
unclassified_count = 0
for i in range(len(questions)):
    if any(questions[i].lower().startswith(x) for x in causal_openers):
        causal_count += 1
    elif any(questions[i].lower().startswith(x) for x in non_causal_openers):
        non_causal_count += 1
    elif any(questions[i].lower().startswith(x) for x in unclassified):
        unclassified_count += 1
print(causal_count / count)
print(non_causal_count/count)
print(unclassified_count/count)


0.020158227119606503
0.5203078599436767
0.08613479231731366


In [344]:
causal_questions = list(compress(questions, indices))
causal_paragraphs = list(compress(paragraphs, indices))
causal_answers = list(compress(answers, indices))

In [417]:
0.020158227119606503 + 0.5203078599436767 + 0.08613479231731366

0.6266008793805968

# Running heuristic on prophetnet output

## Textbook output

In [442]:
def extract_sentences(file_path):
#     with open('textbooktony.csv', 'r') as file:
    with open(file_path, 'r') as file:
        blanks = []
        unnameds = []
        unnamed1s=[]
        IDs = []
        texts = []
        causes = []
        effects = []
        cause_questions = []
        effect_questions = []
        cause_recalls = []
        effect_recalls = []
        reader = csv.reader(file)
        for row in reader:
            blank = row[0]
            unnamed = row[1]
            unnamed1 = row[2]
            ID = row[3]
            text = row[4]
            cause = row[5]
            effect = row[6]
            cause_question = row[7]
            effect_question = row[8]
            cause_recall = row[9]
            effect_recall = row[10]
            blanks.append(blank)
            unnameds.append(unnamed)
            unnamed1s.append(unnamed1)
            IDs.append(ID)
            texts.append(text)
            causes.append(cause)
            effects.append(effect)
            cause_questions.append(cause_question)
            effect_questions.append(effect_question)
            cause_recalls.append(cause_recall)
            effect_recalls.append(effect_recall)
    return blanks, unnameds, unnamed1s, IDs, texts, causes, effects, cause_questions, effect_questions, cause_recalls, effect_recalls
blanks, unnameds, unnamed1s, IDs, texts, causes, effects, cause_questions, effect_questions, cause_recalls, effect_recalls = extract_sentences('CSVs/Textbook/textbook_ce_prophetnet.csv')
cause_questions.extend(effect_questions)

In [443]:
import csv
with open('CSVs/Textbook/textbook_ce_prophetnet.csv') as f:
    reader = csv.DictReader(f)
    print(reader.fieldnames)

['', 'Unnamed: 0', 'Unnamed: 0.1', 'PatternID', 'Text', 'Cause', 'Effect', 'cause_question', 'effect_question', 'cause_recalls', 'effect_recalls']


In [447]:
common_substrings = []
for i in range(len(cause_questions)-1):
    for j in range(i+1, len(cause_questions)):
        common_sentence = longest_common_opener(cause_questions[i], cause_questions[j])
        common_substrings.append(common_sentence)

In [448]:
cause_counts = CountFrequency(common_substrings)
cause_frequencies = sorted(cause_counts.items(),key=operator.itemgetter(1),reverse=True)
cause_frequencies

[('', 15952018),
 ('what ', 3867705),
 ('why ', 1193984),
 ('how ', 483595),
 ('what happens ', 275503),
 ('what is ', 158355),
 ('why is ', 73190),
 ('what is the ', 70408),
 ('why do ', 63000),
 ('what happens when ', 42600),
 ('why are ', 38398),
 ('what happens to ', 28927),
 ('when ', 28594),
 ('why does ', 28501),
 ('what happens if ', 26058),
 ('how can ', 25056),
 ('what can ', 22937),
 ('how does ', 21857),
 ('how do ', 11140),
 ('what does ', 11033),
 ('what happens to the ', 10067),
 ('what causes ', 9564),
 ('why is the ', 8145),
 ('what are ', 8109),
 ('if ', 5633),
 ('why did ', 5532),
 ('how can you ', 5445),
 ('how does the ', 5024),
 ('why does the ', 4510),
 ('what would ', 4425),
 ('is ', 4009),
 ('what did ', 3951),
 ('what do ', 3776),
 ('what does the ', 3558),
 ('why can ', 3475),
 ('how many ', 3298),
 ('what happens when the ', 3214),
 ('when does ', 2523),
 ('what causes the ', 2260),
 ('how is ', 2130),
 ('what are the ', 1976),
 ('what happens if the ', 1871

In [449]:
causal_openers, non_causal_openers, unclassified = classifier(cause_frequencies)

In [450]:
causal_openers = causal_openers[1:]
non_causal_openers = non_causal_openers[1:]
unclassified = unclassified[1:]

In [451]:
count = len(cause_questions)
causal_count = 0
non_causal_count = 0
unclassified_count = 0
for i in range(len(cause_questions)):
    if any(cause_questions[i].lower().startswith(x) for x in causal_openers):
        causal_count += 1
    elif any(cause_questions[i].lower().startswith(x) for x in non_causal_openers):
        non_causal_count += 1
    elif any(cause_questions[i].lower().startswith(x) for x in unclassified):
        unclassified_count += 1
print(causal_count / count)
print(non_causal_count/count)
print(unclassified_count/count)

0.44836309523809526
0.44568452380952384
0.07544642857142857


## Squad output

In [453]:
def extract_sentences(file_path):
#     with open('textbooktony.csv', 'r') as file:
    with open(file_path, 'r') as file:
        blanks = []
        unnameds = []
        IDs = []
        texts = []
        causes = []
        effects = []
        cause_questions = []
        effect_questions = []
        cause_recalls = []
        effect_recalls = []
        reader = csv.reader(file)
        for row in reader:
            blank = row[0]
            unnamed = row[1]
            unnamed1 = row[2]
            unnamed2 = row[3]
            unnamed3 = row[4]
            unnamed4 = row[5]
            ID = row[6]
            text = row[7]
            cause = row[8]
            effect = row[9]
            cause_question = row[10]
            effect_question = row[11]
            cause_recall = row[12]
            effect_recall = row[13]
            blanks.append(blank)
            unnameds.append(unnamed)
            unnameds.append(unnamed1)
            unnameds.append(unnamed2)
            unnameds.append(unnamed3)
            unnameds.append(unnamed4)
            IDs.append(ID)
            texts.append(text)
            causes.append(cause)
            effects.append(effect)
            cause_questions.append(cause_question)
            effect_questions.append(effect_question)
            cause_recalls.append(cause_recall)
            effect_recalls.append(effect_recall)
    return blanks, unnameds, IDs, texts, causes, effects, cause_questions, effect_questions, cause_recalls, effect_recalls
blanks, unnameds, IDs, texts, causes, effects, cause_questions, effect_questions, cause_recalls, effect_recalls = extract_sentences('CSVs/SQuAD/squad_ce_prophetnet.csv')
cause_questions.extend(effect_questions)

In [454]:
import csv
with open('CSVs/SQuAD/squad_ce_prophetnet.csv') as f:
    reader = csv.DictReader(f)
    print(reader.fieldnames)

['', 'Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1', 'Unnamed: 0.1.1.1.1', 'PatternID', 'Text', 'Cause', 'Effect', 'cause_question', 'effect_question', 'cause_recalls', 'effect_recalls']


In [455]:
common_substrings = []
for i in range(len(cause_questions)-1):
    for j in range(i+1, len(cause_questions)):
        common_sentence = longest_common_opener(cause_questions[i], cause_questions[j])
        common_substrings.append(common_sentence)

In [456]:
cause_counts = CountFrequency(common_substrings)
cause_frequencies = sorted(cause_counts.items(),key=operator.itemgetter(1),reverse=True)
cause_frequencies

[('', 1529769),
 ('what ', 474981),
 ('why ', 65695),
 ('how ', 33977),
 ('what is ', 19586),
 ('what is the ', 9675),
 ('what did ', 8314),
 ('why did ', 6014),
 ('when ', 3670),
 ('what happens ', 3370),
 ('why is ', 3042),
 ('what was the ', 2551),
 ('what was ', 2438),
 ('what does ', 2027),
 ('what did the ', 1900),
 ('why did the ', 1542),
 ('who ', 1495),
 ('how did ', 1417),
 ('what are ', 1251),
 ('how do ', 930),
 ('what happens if ', 890),
 ('how can ', 762),
 ('why are ', 757),
 ('what do ', 647),
 ('why was ', 522),
 ('how is ', 479),
 ('why do ', 443),
 ('where ', 348),
 ('why is the ', 348),
 ('why does ', 339),
 ('when did ', 326),
 ('what does the ', 293),
 ('what can ', 290),
 ('how does ', 285),
 ('what happened ', 273),
 ('what happens when ', 257),
 ('what are the ', 231),
 ('why were ', 229),
 ('what has ', 224),
 ('what is a ', 205),
 ('what would ', 199),
 ('what causes ', 174),
 ('is ', 172),
 ('are ', 166),
 ('what are some ', 157),
 ('what happened to ', 156)

In [457]:
causal_openers, non_causal_openers, unclassified = classifier(cause_frequencies)

In [458]:
causal_openers = causal_openers[1:]
non_causal_openers = non_causal_openers[1:]
unclassified = unclassified[1:]

In [459]:
count = len(cause_questions)
causal_count = 0
non_causal_count = 0
unclassified_count = 0
for i in range(len(cause_questions)):
    if any(cause_questions[i].lower().startswith(x) for x in causal_openers):
        causal_count += 1
    elif any(cause_questions[i].lower().startswith(x) for x in non_causal_openers):
        non_causal_count += 1
    else:
        unclassified_count += 1
print(causal_count / count)
print(non_causal_count/count)
print(unclassified_count/count)

0.2767686424474187
0.5889101338432122
0.13432122370936903


## output csv files with causal paragraphs, causal questions, and causal answers for fine-tuning

In [202]:
import csv
def output_csv(causal_questions, causal_paragraphs, causal_answers):
    with open('squad2_training_output.csv','w') as file:
        writer = csv.writer(file)
        writer.writerow(['context', 'question', 'answer', 'answer_starat_idx', 'answer_end_idx'])
        for i in range(len(causal_questions)):
            try:
                print(i)
                writer.writerow([causal_paragraphs[i], causal_questions[i], causal_answers[i]['text'], \
                                 causal_answers[i]['answer_start'], causal_answers[i]['answer_start']+len(causal_answers[i]['text'])])
                file.write('\n')
            except TypeError:
                print('skipped')
                continue
                
#output_csv(causal_questions, causal_paragraphs, causal_answers)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
skipped
65
skipped
66
skipped
67
skipped
68
skipped
69
70
skipped
71
skipped
72
skipped
73
74
skipped
75
skipped
76
77
skipped
78
skipped
79
skipped
80
skipped
81
skipped
82
skipped
83
skipped
84
85
skipped
86
skipped
87
88
89
90
91
skipped
92
skipped
93
94
skipped
95
skipped
96
skipped
97
98
99
100
101
skipped
102
skipped
103
skipped
104
skipped
105
skipped
106
skipped
107
skipped
108
skipped
109
skipped
110
skipped
111
skipped
112
113
skipped
114
115
116
skipped
117
skipped
118
skipped
119
skipped
120
skipped
121
skipped
122
skipped
123
skipped
124
skipped
125
skipped
126
skipped
127
skipped
128
129
skipped
130
131
132
133
134
skipped
135
136
137
skipped
138
skipped
139
skipped
140
141
142
143
skipped
144
145
146
147
148
149
skipped
150
skipped
151
skipped
152
skipped
153
skipped
154
155
156
157
158
15

1224
1225
1226
1227
1228
1229
skipped
1230
skipped
1231
1232
skipped
1233
skipped
1234
1235
skipped
1236
1237
skipped
1238
skipped
1239
1240
1241
skipped
1242
1243
1244
skipped
1245
1246
skipped
1247
1248
skipped
1249
skipped
1250
1251
skipped
1252
skipped
1253
1254
skipped
1255
1256
skipped
1257
1258
1259
skipped
1260
1261
1262
skipped
1263
skipped
1264
1265
skipped
1266
1267
1268
1269
1270
1271
1272
1273
1274
1275
1276
skipped
1277
1278
skipped
1279
skipped
1280
1281
1282
1283
skipped
1284
1285
skipped
1286
skipped
1287
skipped
1288
skipped
1289
1290
1291
skipped
1292
skipped
1293
1294
1295
skipped
1296
1297
skipped
1298
1299
skipped
1300
1301
skipped
1302
1303
skipped
1304
1305
skipped
1306
1307
skipped
1308
1309
1310
skipped
1311
1312
1313
skipped
1314
1315
skipped
1316
1317
skipped
1318
1319
1320
1321
1322
1323
1324
1325
1326
1327
1328
1329
1330
skipped
1331
1332
1333
1334
1335
1336
1337
1338
1339
1340
1341
1342
1343
1344
skipped
1345
skipped
1346
1347
1348
1349
1350
1351
1352
135

2612
2613
skipped
2614
2615
skipped
2616
2617
skipped
2618
2619
skipped
2620
2621
skipped
2622
2623
2624
skipped
2625
2626
2627
2628
2629
skipped
2630
2631
2632
2633
2634
2635
2636
2637
2638
2639
2640
skipped
2641
skipped
2642
2643
2644
skipped
2645
skipped
2646
2647
skipped
2648
2649
skipped
2650
2651
2652
skipped
2653
skipped
2654
2655
skipped
2656
skipped
2657
skipped


In [ ]:
causal_paragraphs[65]

In [198]:
causal_answers[65]

''

In [238]:
squad2_questions = causal_questions

In [251]:
count = 0
for question in causal_questions:
    if question in squad2_questions:
        count += 1
        print('true')
    else:
        print('false')

true
true
true
true
true
false
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
false
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
tru

In [247]:
len(squad2_questions)

2658

In [248]:
len(causal_questions)

1652

In [252]:
count

1645